In [1]:
%load_ext autoreload
%autoreload 2

import torch
from utility.attentionAutoEncoder import AttentionAutoEncoder, concate_data, AttentionDataset
from torch.utils.data import DataLoader

In [2]:
model = AttentionAutoEncoder(3, 5, 2)

In [3]:
input_data = torch.randn(3, 5)
input_data

tensor([[-0.5975, -0.6707, -0.1882, -0.2758,  0.0063],
        [-0.2311,  0.8977, -0.6596, -0.2374, -0.7762],
        [ 0.1455,  0.4327,  0.6082,  1.6839,  0.6171]])

In [4]:
model(input_data)

tensor([[ 0.4820,  6.7751, -6.6022,  4.5491,  7.7067]], grad_fn=<MmBackward0>)

In [5]:
import pandas as pd
from utility.data_processing import set_time_data

In [6]:
dfs = dict()
for ticker in ['btc', 'eth', 'doge', 'xrp', 'ada']:
  currency_metrics = pd.read_csv(f'../data/{ticker}_metrics_5_years.csv')
  set_currency_metrics = set_time_data(currency_metrics)
  dfs[ticker] = set_currency_metrics

In [7]:
res = concate_data(dfs)

In [8]:
X_t = res[:, :, 0]

In [9]:
model(X_t)

tensor([[-0.0183, -0.0017, -0.0749,  0.0514,  0.0709]], grad_fn=<MmBackward0>)

In [10]:
X_t.shape[-1]

5

In [11]:
dataset = AttentionDataset(res)

In [12]:
# Create DataLoader
dataloader = DataLoader(dataset, batch_size=50, shuffle=True, num_workers=0)

# Iterate through DataLoader
for batch in dataloader:
    print(batch.shape)  # Output: torch.Size([10, 5])
    break  # To print only one batch

torch.Size([50, 3, 5])


In [13]:
model(batch)

tensor([[[ 6.3896e-02,  2.3804e-01, -3.4477e-02,  2.4234e-02,  8.3029e-02]],

        [[ 9.1894e-02,  3.2146e-01, -2.2980e-02,  1.6538e-02,  8.9710e-02]],

        [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],

        [[ 1.5300e-01,  4.7494e-01,  3.8499e-02, -2.5312e-02,  6.3668e-02]],

        [[ 2.6258e-02,  1.7666e-01, -1.1458e-01,  7.9091e-02,  1.4622e-01]],

        [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],

        [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],

        [[ 1.7732e-01,  1.0532e+00, -5.9580e-01,  4.1157e-01,  7.8876e-01]],

        [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],

        [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],

        [[ 1.5762e-01,  4.7884e-01,  5.2958e-02, -3.5230e-02,  5.0745e-02]],

        [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],

        [[ 3.7196e-01,  1.2631e+00, -4.4539e-02,  3.3565e-02,  3